## Objectives

-   Read different file types.
-   Extract data from the files.
-   Transform data.
-   Save the transformed data in a ready-to-load format which can be used to load into an RDBMS.


Import the required modules and functions


In [1]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files
from datetime import datetime

## Unzip Files


In [2]:
!unzip datasource.zip -d dealership_data

Archive:  datasource.zip
  inflating: dealership_data/used_car_prices1.csv  
  inflating: dealership_data/used_car_prices2.csv  
  inflating: dealership_data/used_car_prices3.csv  
  inflating: dealership_data/used_car_prices1.json  
  inflating: dealership_data/used_car_prices2.json  
  inflating: dealership_data/used_car_prices3.json  
  inflating: dealership_data/used_car_prices1.xml  
  inflating: dealership_data/used_car_prices2.xml  
  inflating: dealership_data/used_car_prices3.xml  


## About the Data

The file dealership_data contains CSV, JSON, and XML files for used car data which contain features named car_model, year_of_manufacture, price, and fuel.

## Set Paths


In [3]:
data_dir = "dealership_data"
log_file = "logs.txt"
target_file = "prepared_data.csv"

## Extract


### CSV Extract Function


In [4]:
def extract_csv(file_name):
    extracted_data = pd.read_csv(file_name)
    return extracted_data

### JSON Extract Function


In [5]:
def extract_json(file_name):
    extracted_data = pd.read_json(file_name, lines = True)
    return extracted_data

### XML Extract Function


In [6]:
def extract_xml(file_name):
    extracted_data = pd.DataFrame(columns = ["car_model", "year_of_manufacture", "price", "fuel"])
    tree = ET.parse(file_name)
    root = tree.getroot()
    for n, car in enumerate(root):
        car_model = car.find("car_model").text
        year_of_manufacture = int(car.find("year_of_manufacture").text)
        price = float(car.find("price").text)
        fuel = car.find("fuel").text
        car_data = pd.DataFrame({"car_model": car_model, "year_of_manufacture": year_of_manufacture, "price": price, "fuel": fuel}, 
                                index = [n])
        extracted_data = pd.concat([extracted_data, car_data], axis = 0).copy()
    return extracted_data

### Extract Function


In [7]:
def extract(data_dir):
    extracted_data = pd.DataFrame(columns = ["car_model", "year_of_manufacture", "price", "fuel"])
    
    for csv_file in glob.glob(f"{data_dir}/*.csv"):
        csv_data = extract_csv(csv_file)
        extracted_data = pd.concat([extracted_data, csv_data], axis = 0).copy()
        
    for json_file in glob.glob(f"{data_dir}/*.json"):
        json_data = extract_json(json_file)
        extracted_data = pd.concat([extracted_data, json_data], axis = 0).copy()
        
    for xml_file in glob.glob(f"{data_dir}/*.xml"):
        xml_data = extract_xml(xml_file)
        extracted_data = pd.concat([extracted_data, xml_data], axis = 0).copy()
    
    return extracted_data

## Transform


In [8]:
def transform(data):
    data["price"] = round(data["price"], 2)
    return data

## Loading


In [9]:
def load(file_name, data):
    data.to_csv(file_name)

## Logging


In [10]:
def log(message):
    timestamp_format = "%H:%M:%S-%H-%d-%Y"
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    
    with open(log_file, "a") as file:
        file.write(f"{timestamp} - {message.upper()}\n")

## Running ETL Process


In [11]:
log("ETL job started")
log("Extract phase Started")
extracted_data = extract(data_dir)
log("Extract phase Ended")

log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")

log("Load phase Started")
load(target_file, transformed_data)
log("Load phase Ended")
log("ETL job ended")